In [1]:
import torch
import random
import numpy as np
from PIL import Image
from datasets import load_dataset
from IPython.display import Audio
from diffusers import AutoencoderKL, AudioDiffusionPipeline, Mel

2023-05-26 06:09:53.858068: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-26 06:09:53.886487: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 06:09:54.451547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mel = Mel()
vae = AutoencoderKL.from_pretrained('../models/autoencoder-kl')
# vae = AutoencoderKL.from_pretrained('/home/genzorr/Documents/learning/Skoltech/dl/audio-diffusion-mod/lightning_logs/version_0/checkpoints/epoch=4-step=90.ckpt')
# vae = AutoencoderKL()
# vae.load_state_dict(state_dict=torch.load('/home/genzorr/Documents/learning/Skoltech/dl/audio-diffusion-mod/lightning_logs/version_0/checkpoints/epoch=4-step=90.ckpt')['state_dict'])

In [3]:
vae.config

FrozenDict([('in_channels', 1),
            ('out_channels', 1),
            ('down_block_types',
             ['DownEncoderBlock2D',
              'DownEncoderBlock2D',
              'DownEncoderBlock2D',
              'DownEncoderBlock2D']),
            ('up_block_types',
             ['UpDecoderBlock2D',
              'UpDecoderBlock2D',
              'UpDecoderBlock2D',
              'UpDecoderBlock2D']),
            ('block_out_channels', [128, 256, 512, 512]),
            ('layers_per_block', 2),
            ('act_fn', 'silu'),
            ('latent_channels', 1),
            ('norm_num_groups', 32),
            ('sample_size', [256, 256]),
            ('scaling_factor', 0.18215),
            ('_class_name', 'AutoencoderKL'),
            ('_diffusers_version', '0.16.1'),
            ('_name_or_path', '../models/autoencoder-kl')])

In [4]:
ds = load_dataset('teticio/audio-diffusion-256')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ds = load_dataset('teticio/audio-diffusion-256')                                             │
│   2                                                                                              │
│                                                                                                  │
│ /home/genzorr/Documents/learning/Skoltech/dl/.venv/lib/python3.10/site-packages/datasets/load.py │
│ :1797 in load_dataset                                                                            │
│                                                                                                  │
│   1794 │   try_from_hf_gcs = path not in _PACKAGED_DATASETS_MODULES                              │
│   1795 │                                                                                         │
│   1796 │   # Download and prepare data                                                           │
│ ❱ 1797 │   builder_instance.download_and_prepare(                                                │
│   1798 │   │   download_config=download_config,                                                  │
│   1799 │   │   download_mode=download_mode,                                                      │
│   1800 │   │   verification_mode=verification_mode,                                              │
│                                                                                                  │
│ /home/genzorr/Documents/learning/Skoltech/dl/.venv/lib/python3.10/site-packages/datasets/builder │
│ .py:890 in download_and_prepare                                                                  │
│                                                                                                  │
│    887 │   │   │   │   │   │   │   prepare_split_kwargs["max_shard_size"] = max_shard_size       │
│    888 │   │   │   │   │   │   if num_proc is not None:                                          │
│    889 │   │   │   │   │   │   │   prepare_split_kwargs["num_proc"] = num_proc                   │
│ ❱  890 │   │   │   │   │   │   self._download_and_prepare(                                       │
│    891 │   │   │   │   │   │   │   dl_manager=dl_manager,                                        │
│    892 │   │   │   │   │   │   │   verification_mode=verification_mode,                          │
│    893 │   │   │   │   │   │   │   **prepare_split_kwargs,                                       │
│                                                                                                  │
│ /home/genzorr/Documents/learning/Skoltech/dl/.venv/lib/python3.10/site-packages/datasets/builder │
│ .py:963 in _download_and_prepare                                                                 │
│                                                                                                  │
│    960 │   │   # Generating data for all splits                                                  │
│    961 │   │   split_dict = SplitDict(dataset_name=self.name)                                    │
│    962 │   │   split_generators_kwargs = self._make_split_generators_kwargs(prepare_split_kwarg  │
│ ❱  963 │   │   split_generators = self._split_generators(dl_manager, **split_generators_kwargs)  │
│    964 │   │                                                                                     │
│    965 │   │   # Checksums verification                                                          │
│    966 │   │   if verification_mode == VerificationMode.ALL_CHECKS and dl_manager.record_checks  │
│                                                                                                  │
│ /home/genzorr/Documents/learning/Skoltech/dl/.venv/lib/pyth

### Reconstruct audio

In [ ]:
image = random.choice(ds['train'])['image']
display(image)
Audio(data=mel.image_to_audio(image), rate=mel.get_sample_rate())

In [ ]:
# encode
input_image = np.frombuffer(image.tobytes(), dtype="uint8").reshape(
    (image.height, image.width, 1))
input_image = ((input_image / 255) * 2 - 1).transpose(2, 0, 1)
posterior = vae.encode(torch.tensor([input_image],
                                    dtype=torch.float32)).latent_dist
latents = posterior.sample()

In [ ]:
# reconstruct
output_image = vae.decode(latents)['sample']
output_image = torch.clamp(output_image, -1., 1.)
output_image = (output_image + 1.0) / 2.0  # -1,1 -> 0,1; c,h,w
output_image = (output_image.detach().cpu().numpy() *
                255).round().astype("uint8").transpose(0, 2, 3, 1)[0, :, :, 0]
output_image = Image.fromarray(output_image)
display(output_image)
Audio(data=mel.image_to_audio(output_image), rate=mel.get_sample_rate())

### Random sample from latent space
(Don't expect interesting results!)

In [ ]:
# sample
output_image = vae.decode(torch.randn_like(latents))['sample']
output_image = torch.clamp(output_image, -1., 1.)
output_image = (output_image + 1.0) / 2.0  # -1,1 -> 0,1; c,h,w
output_image = (output_image.detach().cpu().numpy() *
                255).round().astype("uint8").transpose(0, 2, 3, 1)[0, :, :, 0]
output_image = Image.fromarray(output_image)
display(output_image)
Audio(data=mel.image_to_audio(output_image), rate=mel.get_sample_rate())

### Interpolate between two audios in latent space

In [ ]:
image2 = random.choice(ds['train'])['image']
display(image2)
Audio(data=mel.image_to_audio(image2), rate=mel.get_sample_rate())

In [ ]:
# encode
input_image2 = np.frombuffer(image2.tobytes(), dtype="uint8").reshape(
    (image2.height, image2.width, 1))
input_image2 = ((input_image2 / 255) * 2 - 1).transpose(2, 0, 1)
posterior2 = vae.encode(torch.tensor([input_image2],
                                     dtype=torch.float32)).latent_dist
latents2 = posterior2.sample()

In [ ]:
# interpolate
alpha = 0.5  #@param {type:"slider", min:0, max:1, step:0.1}
output_image = vae.decode(
    AudioDiffusionPipeline.slerp(latents, latents2, alpha))['sample']
output_image = torch.clamp(output_image, -1., 1.)
output_image = (output_image + 1.0) / 2.0  # -1,1 -> 0,1; c,h,w
output_image = (output_image.detach().cpu().numpy() *
                255).round().astype("uint8").transpose(0, 2, 3, 1)[0, :, :, 0]
output_image = Image.fromarray(output_image)
display(output_image)
display(Audio(data=mel.image_to_audio(image), rate=mel.get_sample_rate()))
display(Audio(data=mel.image_to_audio(image2), rate=mel.get_sample_rate()))
display(
    Audio(data=mel.image_to_audio(output_image), rate=mel.get_sample_rate()))